# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):  
    # Join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [1]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Create Cluster and Keyspace 

### Creating a Cluster

In [2]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

### Create Keyspace

In [3]:
session.execute("""
CREATE KEYSPACE IF NOT EXISTS events
WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}
""")

### Set Keyspace

In [4]:
session.set_keyspace('events')

## Create queries to ask the following three questions of the data

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### Query 1
This first query expects as output the `artist`, the `song` name and the `length` of the song, which was listened is a specific session (`sessionId`) in a specific order (`itemInSession`). 

Therefore, the columns in the table created and modelled to answer this specific question/querry, will be: `artist`, `song`, `sessionId`, `itemInSession` and `length`, with `sessionId` and `itemInSession` as **PRIMARY KEYS**.

#### Creating the table

In [5]:
create = """
CREATE TABLE IF NOT EXISTS sessions (sessionId int, 
                                     itemInSession int,
                                     artist text, 
                                     song text, 
                                     length float,
                           PRIMARY KEY (sessionId, itemInSession))
"""

session.execute(create)

#### Adding data into the table

In [6]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO sessions (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Verifying that the data have been correctly inserted into the table

In [7]:
select = """
SELECT artist, song, length 
FROM sessions 
WHERE sessionId = 338 
    AND itemInSession = 4
"""

results = session.execute(select)

for result in results:
    print(result.artist, result.song, result.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query 2
`Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182`
This second query expects the `artist` name, the `song` name and the `firstName` and `lastName` of the user that corresponds to a specific `userId` and `sessionID`.

Therefore, the columns in the table created and modelled to answer this specific question/querry, will be: `artist`, `song`, `firstName`, `lastName`, `sessionId`, `userId`, and `itemInSession`. In order to establish a correct **PARTITION KEY**, both `sessionId` and `userId` will be used, to ensure the uniqueness of each record and avoid conflicting issues in the database. Plus, `itemInSession` will be included as A **CLUSTERING COLUMN** to be included in the _WHERE_ statement used in the _SELECT_ query.

#### Creating the table

In [9]:
create = """
CREATE TABLE IF NOT EXISTS users (sessionId int,
                                  userId int,
                                  itemInSession int,
                                  artist text, 
                                  song text, 
                                  firstName text,
                                  lastName text,
                           PRIMARY KEY ((sessionId, userId), itemInSession))
"""

session.execute(create)                   

#### Adding data into the table

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users (sessionId, userId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[10]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Verifying that the data have been correctly inserted into the table

In [11]:
select = """
SELECT artist, song, firstName, lastName
FROM users
WHERE sessionId = 182
    AND userId = 10
"""

results = session.execute(select)

for result in results:
    print(result.artist, result.song, result.firstname, result.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3
The third and final query expects as output the users' `firsName` and `lastName` that listened to a specific `song`. 

This means that to answer this specific question/querry, a table will have to be created and modelled aiming at it, including the following columns: `song`, `firstName`, `lastName` and `userId`. Both `song` and `userId` will act as **PARTITION** and **PRIMARY KEYS**.

#### Creating the table

In [12]:
create = """
CREATE TABLE IF NOT EXISTS songs (song text,
                                  userId int,
                                  firstName text,
                                  lastName text,
                           PRIMARY KEY (song, userId))
"""

session.execute(create)

#### Adding data into the table

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Verifying that the data have been correctly inserted into the table

In [16]:
select = """
SELECT firstName, lastName
FROM songs
WHERE song = 'All Hands Against His Own'
"""

results = session.execute(select)

for result in results:
    print(result.firstname, result.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


## Drop the tables before closing out the sessions

In [74]:
drops = ["DROP TABLE sessions",
        "DROP TABLE users",
        "DROP TABLE songs"]

for drop in drops:
    try:
        session.execute(drop)
    except:
        pass

## Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()